## LLaVA (Large Language and Vision Assistant)

* End-to-End Trained Large Multi-Modal Model that connects a Vision Encoder (CLIP) and an LLM (Vicuna) for general purpose visual and language understanding.

![img](https://images.prismic.io/encord/b81c019b-5d0a-44eb-8a6c-f6ccb4f3e24a_image4.png?auto=compress,format)

#### 1. Vision Encoder: At the foundation of LLaVa’s architecture is the pre-trained CLIP visual encoder, specifically the ViT-L/14 variant. This component processes input images (Xv) through Transformer layers to extract features (Zv), enabling the model to understand visual information effectively.

#### 2. Language Model (Vicuna): LLaVa’s linguistic capabilities rely on Vicuna, a variant of a large language model (LLM) denoted by fϕ . Vicuna comprehends and generates language responses (Xa) based on input language instructions (Xq), complementing the vision encoder’s functionality.

#### 3. Linear Projection: This component, represented by a trainable matrix (W), serves as the bridge between visual features (Zv) and the language model’s embedding space. It transforms visual features into visual tokens (Hv), aligning them with the language model’s word embedding space to facilitate multimodal conversation

## Data Prep

2 steps involved:

### caption = brief description

* Use existing image caption datasets like CC3M
* with caption as an input, create a list of varied questions answers. Use this dataset to create instructions of the format:

"""    
Human: \<question\> \<image\>    
Assistant: \<caption\>    
"""


In [ ]:
{
    "id": "002239345",
    "image": "00223/002239345.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "Write a terse but informative summary of the picture.\n<image>"
      },
      {
        "from": "gpt",
        "value": "a grey watch with an army style strap"
      }
    ]
  }

### gpt-4 = reasoning

* Image caption instruction data mentioned above lacks in-depth reasoning and diversity. In order to mitigate this authors use GPT4 to create data by just using captions as an input. Below is the prompt mentioned in the paper

![img](https://imgur.com/zdkvz98.png)

GPT4 is used for:

* Conversational data: Conversation between assistant and person as as if assistant is seeing image.
* Detailed description: Generate detailed description of image using sample set of questions.
* Complex reasoning: Generate complex questions where answering requires logical breakdown of question and multiple steps to arrive at final answer.

## Architecture

LLaVA utilizes 2 pretrained models in their architecture:

1. A Pre-Trained Language Model: Vicuna 13B - which is a fine tuned derivative of Llama2 13B variant
2. CLIP Vision Encoder

## Stage 1: Pretraining for feature alignment

#### CLIP embeddings are present in a different hyperspace, plus they cannot be used as an input to LLMs readily because of different dimensionality. Hence to solve for this a projection of these embeddings is obtained using Linear layers. In this stage we keep CLIP embeddings and LLM weights frozen and only learn the weights of this Linear layer.

#### During this stage, a visual encoder (such as the CLIP visual encoder ViT-L/14) processes the images to extract visual features, and a projection matrix (W) is then used to map these features into the word embedding space of the language model.

![img](https://imgur.com/T1v2Gb1.png)

For training projection layer filtered CC3M image text pairs dataset was used. Each sample was treated as single turn conversation. Learning task was to predict caption given image and a question (generated by GPT4) associated with it.

Sample of pretraining data:

![img](https://imgur.com/9m3eMmj.png)

## Stage 2 : Fine tuning end to end

#### Visual encoder layers are kept frozen, only projection layer and LLM weights are learnt in this stage. Here the instruction tuning data is used conversation, complex reasoning and description instructions which were created using GPT4. As a result of this a multimodal chatbot is obtained.

#### After aligning the visual and language features, LLaVA undergoes an end-to-end fine-tuning process. Despite keeping the visual encoder’s weights frozen, this stage allows the model to fine-tune the weights of the projection matrix and language model jointly. The objective is to maximize the likelihood of the target answers based on the multimodal data provided.

#### Stage 2 --  ensures that the model does not just understand images in the context of generic descriptions but can engage in complex dialogues, provide detailed explanations, and reason through problems when prompted with specific questions related to the images.

### LLaVA model architecture



```python
LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
		...

      (post_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (multi_modal_projector): LlavaMultiModalProjector(
    (linear_1): Linear(in_features=1024, out_features=4096, bias=True)
    (act): GELUActivation()

    (linear_2): Linear(in_features=4096, out_features=4096, bias=True)
    # ^^^^^^^^^^^^ note
  )
  (language_model): LlamaForCausalLM(
    (model): LlamaModel(

      (embed_tokens): Embedding(32064, 4096)
      # ^^^^^^^^^^^^ note
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
		...

      )
      (norm): LlamaRMSNorm()
    )
    (lm_head): Linear(in_features=4096, out_features=32064, bias=False)
  )
)
```

![img](https://imgur.com/ALWJ7Mt.png)

## Eyes Wide Shut? Exploring the Visual Shortcomings of Multimodal LLMs - https://arxiv.org/abs/2401.06209

- Talks about Design Choices in making the Visual Part of LLaVA stronger
